In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Reshape, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, MeanSquaredError
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Hàm tính IOU
def compute_iou(box1, box2):
    x1 = np.maximum(box1[0], box2[0])
    y1 = np.maximum(box1[1], box2[1])
    x2 = np.minimum(box1[2], box2[2])
    y2 = np.minimum(box1[3], box2[3])
    
    intersection_area = np.maximum(x2 - x1, 0) * np.maximum(y2 - y1, 0)
    
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    union_area = box1_area + box2_area - intersection_area
    
    iou = intersection_area / union_area
    return iou

# Tạo mô hình VGG16 backbone
def create_vgg16_backbone(input_shape=(256, 256, 3)):
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in vgg16.layers:
        layer.trainable = False
    return vgg16

# Tạo RPN (Region Proposal Network)
def create_rpn(backbone):
    rpn_conv = Conv2D(256, (3, 3), activation='relu', padding='same')(backbone.output)
    rpn_cls = Conv2D(9, (1, 1), activation='sigmoid', name='rpn_cls')(rpn_conv)
    rpn_reg = Conv2D(36, (1, 1), activation='linear', name='rpn_reg')(rpn_conv)
    
    rpn_cls_flat = Flatten()(rpn_cls)
    rpn_reg_flat = Flatten()(rpn_reg)
    
    return rpn_cls_flat, rpn_reg_flat

# Tạo lớp đầu ra cuối cùng cho classification và regression
def create_detection_head(backbone, num_classes):
    x = GlobalAveragePooling2D()(backbone.output)
    cls_output = Dense(num_classes, activation='softmax', name='cls_output')(x)
    reg_output = Dense(4, activation='linear', name='reg_output')(x)
    return cls_output, reg_output

# Tạo mô hình Faster R-CNN với VGG16 backbone, RPN và detection head
def create_faster_rcnn_model(input_shape=(256, 256, 3), num_classes=5):
    input_layer = Input(shape=input_shape)
    backbone = create_vgg16_backbone(input_shape)(input_layer)
    rpn_cls_pred, rpn_reg_pred = create_rpn(backbone)
    cls_output, reg_output = create_detection_head(backbone, num_classes)
    
    rcnn_model = Model(inputs=input_layer, outputs=[rpn_cls_pred, rpn_reg_pred, cls_output, reg_output])
    return rcnn_model

# Chuẩn bị dữ liệu
train_dir = 'O:/Rcnn_Plant_diseases/Dataset/train'
valid_dir = 'O:/Rcnn_Plant_diseases/Dataset/valid'

train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='sparse'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='sparse'
)

# Xây dựng mô hình Faster R-CNN
rcnn_model = create_faster_rcnn_model(input_shape=(256, 256, 3), num_classes=train_generator.num_classes)

# Compile mô hình với optimizer và loss function
rcnn_model.compile(optimizer=Adam(learning_rate=0.0001),
                   loss={'rpn_cls': 'binary_crossentropy', 'rpn_reg': MeanSquaredError(),
                         'cls_output': SparseCategoricalCrossentropy(), 'reg_output': MeanSquaredError()},
                   metrics={'rpn_cls': 'accuracy', 'rpn_reg': 'mse', 'cls_output': 'accuracy', 'reg_output': 'mse'})

# Huấn luyện mô hình
rcnn_history = rcnn_model.fit(train_generator, epochs=10, validation_data=valid_generator)

# Lưu mô hình
rcnn_model.save('faster_rcnn_model.h5')

# Lịch sử huấn luyện được trả về từ phương thức fit()
history = rcnn_history.history

# Lưu lịch sử huấn luyện vào một tệp JSON
import json
with open('training_history.json', 'w') as f:
    json.dump(history, f)


Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.
Epoch 1/10


ValueError: In the dict argument `loss`, key 'rpn_cls' does not correspond to any model output. Received:
loss={'rpn_cls': <keras.src.losses.losses.SparseCategoricalCrossentropy object at 0x0000025878D06AE0>, 'rpn_reg': 'mse'}

In [ ]:
# Huấn luyện mô hình
rcnn_history = rcnn_model.fit(train_generator, epochs=10, validation_data=valid_generator)


Epoch 1/10


h:\Applications\Envir\Pyth\Lib\site-packages\keras\src\optimizers\base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(
h:\Applications\Envir\Pyth\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 173/2197 ━━━━━━━━━━━━━━━━━━━━ 26:31 786ms/step - functional_1_accuracy: 0.0012 - loss: 11.8172